Watch DM progress

In [1]:
import os
import time
import numpy as np
import xarray as xr
result_dir = '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/'
path = os.path.join(result_dir, 'samples')

In [4]:

in_ds = xr.load_dataset(path + '/uq_modal_beam.nc')
# out_ds = xr.load_dataset(path + '/uq_modal_beam_out.nc')
# np.isnan(out_ds._runtimes.data)[0][0]

In [ ]:
for i in range(3999242,7599653): # 3938021 is the first invalid # 3625888 is broken, is not broken?
    if not i%250:
        print('.', end='', flush=True)
    jid = in_ds.ids[i].item()
    res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
    arr = np.load(res_dir, allow_pickle=True)
    assert arr['self.struct_parms'].item()['dy_tmd'] == in_ds['dD'][i].item()

In [5]:
display(i)

3999242

In [38]:
jid = in_ds.ids[i].item()
res_dir = os.path.join(path,jid,f'{jid}_mechanical.npz')
print(time.ctime(os.stat(res_dir).st_mtime))
print(time.ctime(os.stat(res_dir).st_ctime))

Mon Nov  7 09:21:49 2022
Mon Nov  7 09:21:49 2022


- 9,999,693 = 729 * 13,717 # total samples
- 3,930,727 / 729 = 5,391.9 # last datamanager sample with runtime not nan -> start_ale=5390 is the last good sample when plotting in estimate_imp
- 6,581,412 = 729 * 9,028 # last datamanager sample with results not nan
- 3,938,021 / 729 = 5,401.9 # last sample, that progress watcher marks valid, probably the last 7294 samples were not saved into db -> it is bad when plotting in estimate_imp

In [ ]:
for ret_name in ['damp_freqs','zetas','frf']:
    
    if ret_name == 'frf':
        # continue
        inds = range(1025*2)
    else:
        # continue
        inds = range(14)
    all_times = []
    all_dirs = []
    all_progr = []
    for ind in inds:
        if ret_name == 'frf':
            ret_ind = {'frequencies':ind//2, 'space':ind%2}
            if not ind%2: continue
        else:
            ret_ind = {'modes':ind}
            
        ret_dir = f'{ret_name}-{".".join(str(e) for e in ret_ind.values())}'
        fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
        # print(ret_dir)
        if os.path.exists(fname):
            all_times.append(os.stat(fname).st_mtime)
            all_dirs.append(ret_dir)
            try:
                intp_err = np.load(fname)['self.intp_errors']
            except:
                all_progr.append(0)
                continue
            nan_ind = np.where(np.isnan(intp_err))[0]
            if len(nan_ind):
                all_progr.append(np.min(nan_ind))
            else:
                all_progr.append(len(intp_err))
        else:
            all_progr.append(0)
            all_dirs.append(ret_dir)
            all_times.append(0)
    
    display(f'finished: {np.sum(np.array(all_progr)==5390)}, not finished {np.sum(np.array(all_progr)!=5390)}')
    for ret_time,ret_dir, progr in sorted(zip(all_times, all_dirs, all_progr)):
        # if progr == 5390: continue
        # if 1675933971.0 <= ret_time <= 1675934235.0 and progr==0: continue
        print(f'{ret_dir} @ {progr} on {time.ctime(ret_time)} {ret_time}')

In [67]:
ret_dir = 'frf-585.0'
fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
os.stat(fname).st_mtime > 1675778682.0

True

In [ ]:
ret_name = 'frf'
for freq_ind in range(1025):
    ret_ind = {'frequencies':freq_ind, 'space':0}
    ret_dir = f'{ret_name}-{".".join(str(e) for e in ret_ind.values())}'
    fname = os.path.join(result_dir, 'estimations', f'{ret_dir}/polyuq_imp.npz')
    
    if os.path.exists(fname):
        if os.stat(fname).st_mtime < 1675932359.0:
            os.rename(fname, fname + '.bak')
        else:
            
            print(time.ctime(os.stat(fname).st_mtime))
    else:
        print(ret_dir)
        f = open(fname,'wb')
        f.close()

In [ ]:
import shutil
N_mcs_ale = 13717 # N_mcs = 1e6 = N_mcs_ale * N_mcs_epi
N_mcs_epi = 729 # = 3^6 = 2.56^n_imp ~ 3^n_imp -> cover every corner and midpoints in a full-factorial design (but distributed)
result_dir = os.path.join('/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/', 'samples')
ale_dirs = []

In [10]:
for jid in in_ds.ids:
    jid = jid.data.item()
    id_ale, id_epi = jid.split('_')
    
    ale_dir = os.path.join(result_dir,str(id_ale))
    if not ale_dir in ale_dirs:
        print(f'creating {ale_dir} total {len(ale_dirs)}')
        assert not os.path.exists(ale_dir)
        os.makedirs(ale_dir)
        assert os.path.exists(ale_dir)
        ale_dirs.append(ale_dir)
    elif ale_dir!=ale_dirs[-1]:
        continue
        
    srcfolder = os.path.join(result_dir, jid)
    dstfolder = os.path.join(ale_dir, str(id_epi))
    try:
        shutil.move(srcfolder,dstfolder)
    except Exception as e:
        print(e)
    

[Errno 2] No such file or directory: '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/samples/98518f2a1683_7dd88f90876c'
[Errno 2] No such file or directory: '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/samples/98518f2a1683_2efd9a68b225'
[Errno 2] No such file or directory: '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/samples/98518f2a1683_0fb585470117'
[Errno 2] No such file or directory: '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/samples/98518f2a1683_78f1ecff12f4'
[Errno 2] No such file or directory: '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/samples/98518f2a1683_7f6b239378d3'
[Errno 2] No such file or directory: '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/samples/98518f2a1683_7ff2a29516bd'
[Errno 2] No such file or directory: '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/samples/98518f2a1683_311fe2562b6c'
[Errno 2] No such file or directory: '/usr/scratch4/sima9999/work/modal_uq/uq_modal_beam/samples/98518f2a1683_f5aaeea4c2b0'
[Errno 2